In [1]:
import gym
import numpy as np 
import matplotlib.pyplot as plt
import warnings
import highway_env
warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
#Create the Model
def create_model():
    model= tf.keras.models.Sequential()

    model.add(tf.keras.layers.Flatten(input_shape=(1,5,5)))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))    
    model.add(tf.keras.layers.Dense(32, activation='relu')) 
    
    model.add(tf.keras.layers.Dense(5, activation='linear'))
    
    return model

In [4]:
env= gym.make('highway-v0')

In [5]:
env.reset()

array([[ 1.        ,  1.        ,  0.5       ,  0.41666666,  0.        ],
       [ 1.        ,  0.12953883, -0.5       , -0.05355501,  0.        ],
       [ 1.        ,  0.26911008, -0.5       , -0.04965264,  0.        ],
       [ 1.        ,  0.39865765, -0.25      , -0.02877636,  0.        ],
       [ 1.        ,  0.53819424,  0.25      , -0.03109277,  0.        ]],
      dtype=float32)

In [6]:
env.config['vehicles_density']=2
env.reset()

array([[ 1.        ,  1.        ,  0.        ,  0.41666666,  0.        ],
       [ 1.        ,  0.07794033,  0.75      , -0.01801931,  0.        ],
       [ 1.        ,  0.14426376,  0.25      , -0.02270218,  0.        ],
       [ 1.        ,  0.21243836,  0.        , -0.02724218,  0.        ],
       [ 1.        ,  0.27671912,  0.5       , -0.05047384,  0.        ]],
      dtype=float32)

In [7]:
env.config['vehicles_density']

2

In [8]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [9]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=75000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=5, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
# model= create_model()
# dqn = build_agent(model, 5)
# dqn.compile(tf.keras.optimizers.Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)
dqn.save_weights('dqn_weights_dense_latest2.h5f')

Training for 5000 steps ...
Interval 1 (0 steps performed)
  266/10000 [..............................] - ETA: 1:18:08 - reward: 0.6682

In [22]:
model= create_model()
dqn = build_agent(model, 5)
dqn.compile(tf.keras.optimizers.Adam(lr=1e-3), metrics=['mae'])

# dqn.load_weights('dqn_weights.h5f')          #All it wants is right lane                  25-->25-->5
# dqn.load_weights('dqn_weights2.h5f')         #Doesn't understand slowing down notion.     25-->128-->128-->64-->32-->5
dqn.load_weights('dqn_weights_dense_latest.h5f')     #Doesn't understand slowing down notion.      25-->128-->128-->64-->32-->5
# dqn.load_weights('dqn_weights_dense2.h5f')   #Worst, doesn't do anything.                 25-->256-->256-->5
# dqn.load_weights('dqn_weights_dense007.h5f') #Very bad, doesn't do anything               25-->128-->128-->64-->32-->5


In [23]:
dqn.test(env,nb_episodes=3,visualize=True)
env.close()

Testing for 3 episodes ...
Episode 1: reward: 22.294, steps: 34
Episode 2: reward: 28.756, steps: 40
Episode 3: reward: 21.827, steps: 30
